In [1]:
from bs4 import BeautifulSoup as bs
import os
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import glob
import seaborn as sns
import plotly.express as px
import descartes
import re
from datetime import datetime
import calmap
import geopandas
from pyspark import *
from pyspark.sql import *
from pyspark.sql.functions import *
import re
from pyspark.sql.types import ArrayType
from pyspark.sql.types import StringType
#import geoplot
#import shapefile as shp
import string

from sklearn.linear_model import LinearRegression, LogisticRegression
import geopandas
import descartes
import geoplot
import shapefile as shp
import warnings; warnings.simplefilter('ignore')

In [2]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
df = spark.read.option("header", "true").csv("data/cat_csv/*.csv")
df.count()
agora_vendors_df = df.select('vendor', 'date').distinct().toPandas()

In [3]:
SR_vendors_df = pd.read_csv("data/total.dat")
evo_vendors_df = pd.read_csv("data/evo_vendors.txt")
c9_vendors_df = pd.read_csv("data/c9_vendors.txt")

In [4]:
def cleanName(s):
    return str(s).translate(string.punctuation).lower()

In [5]:
agora_vendors_df['vendor']    = agora_vendors_df['vendor'].apply(cleanName)
SR_vendors_df['vendor']  = SR_vendors_df['vendor'].apply(cleanName)
evo_vendors_df['vendor'] = evo_vendors_df['vendor'].apply(cleanName)
c9_vendors_df['vendor']  = c9_vendors_df['vendor'].apply(cleanName)
fanfou_df['vendor'] = fanfou_df['vendor'].apply(cleanName)

In [6]:
agora_min_df = agora_vendors_df.groupby(by='vendor').min()
agora_max_df = agora_vendors_df.groupby(by='vendor').max()

In [7]:
evo_vendors_df['exact_Evo'] = 1
evo_min_df = evo_vendors_df.groupby(by='vendor').min()
evo_max_df = evo_vendors_df.groupby(by='vendor').max()

In [8]:
SR_vendors_df = pd.DataFrame.drop_duplicates(SR_vendors_df)
c9_vendors_df = pd.DataFrame.drop_duplicates(c9_vendors_df)

In [9]:
SR_vendors_df['exact_SR'] = 1
c9_vendors_df['exact_c9'] = 1

In [10]:
central_df = pd.DataFrame.drop_duplicates(agora_vendors_df.drop(columns=['date']))
central_df = central_df.set_index('vendor').join(agora_min_df).join(agora_max_df, lsuffix='_min', rsuffix='_max')

In [11]:
central_df = central_df.join(evo_min_df, how='left').join(evo_max_df, how='left', lsuffix='_evo_min', rsuffix='_evo_max').rename(columns={'exact_Evo_evo_max' : 'exact_Evo'}).drop(columns=['exact_Evo_evo_min'])

In [12]:
central_df = central_df.join(SR_vendors_df.set_index('vendor'), how='left').join(c9_vendors_df.set_index('vendor'), how='left')

In [13]:
central_df.reset_index(inplace=True)

In [14]:
import hashlib


In [15]:
def hamming_distance(chaine1, chaine2):
    if (len(chaine1) < 4) & (len(chaine1) != len(chaine2)):
        return 100
    return np.sum(c1 != c2 for c1, c2 in zip(chaine1, chaine2)) + np.abs((len(chaine1)-len(chaine2)))


def isThereAlmost(name, df):
    res = df['vendor'].apply(lambda x : hamming_distance(x, name))
    if len(res[res < 2]) > 0:
       # print(name)
       # print(res[res < 2])
        return 1
    else:
        return 0

In [16]:
central_df['hamming_SR'] = central_df['vendor'].apply(lambda x : isThereAlmost(x, SR_vendors_df))

In [17]:
central_df['hamming_evo'] = central_df['vendor'].apply(lambda x : isThereAlmost(x, evo_min_df.reset_index()))

In [18]:
central_df['hamming_c9'] = central_df['vendor'].apply(lambda x : isThereAlmost(x, c9_vendors_df))

In [19]:
survivor_all_mkt = central_df[(central_df['date_min'] < '2014-11-07') & (central_df['date_max'] > '2014-11-10')].dropna()
dead_all_mkt = central_df[(central_df['date_min'] < '2014-11-07') & (central_df['date_max'] < '2014-11-10')].dropna()

In [62]:
pd.DataFrame.drop_duplicates(central_df.reset_index())

,vendor,exact_Evo,exact_SR,exact_c9,hamming_SR,hamming_evo,hamming_c9,three_mkt,h_three_mkt,two_mkt,h_two_mkt,all_mkt,h_all_mkt,ag_and_dead_market,ag_and_evo_market,h_ag_only,date_min,date_max
0,"""<i class=""""fa fa-t""",NaN,NaN,NaN,0,0,0,NaN,0,NaN,0,NaN,0,0,0,1,NaN,NaN
1,-bigg-balls-,1.0,1.0,1.0,1,1,1,1.0,1,1.0,1,1.0,1,1,1,0,2014-09-22,2014-09-22
2,-euphoria-,NaN,NaN,NaN,0,0,0,NaN,0,NaN,0,NaN,0,0,0,1,2015-04-02,2015-07-07
3,-flip-,NaN,NaN,NaN,0,0,0,NaN,0,NaN,0,NaN,0,0,0,1,2014-12-10,2015-03-19
4,-inanna-,NaN,NaN,NaN,0,0,0,NaN,0,NaN,0,NaN,0,0,0,1,2014-02-02,2015-07-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3220,zya,NaN,NaN,NaN,0,0,0,NaN,0,NaN,0,NaN,0,0,0,1,2014-01-09,2014-05-24
3221,~bigboy,NaN,NaN,NaN,0,0,0,NaN,0,NaN,0,NaN,0,0,0,1,2014-08-03,2015-07-07
3222,~flatbushzombies~,NaN,NaN,NaN,0,0,0,NaN,0,NaN,0,NaN,0,0,0,1,2014-04-06,2014-04-06
3223,~luxurytransport,NaN,NaN,NaN,0,0,0,NaN,0,NaN,0,NaN,0,0,0,1,2015-06-15,2015-06-15


In [21]:
central_df.to_csv('data/market_per_v_count.csv', index=None)

In [22]:
central_df['three_mkt'] = central_df['exact_Evo'] * central_df['exact_SR'] + central_df['exact_Evo'] * central_df['exact_c9'] + central_df['exact_SR'] *central_df['exact_c9']
central_df['three_mkt'] = central_df['three_mkt'].apply(lambda x:  1 if x > 1 else x)

In [23]:
central_df['h_three_mkt'] = central_df['hamming_evo'] * central_df['hamming_SR'] + central_df['hamming_evo'] * central_df['hamming_c9'] + central_df['hamming_SR'] *central_df['hamming_c9']
central_df['h_three_mkt'] = central_df['h_three_mkt'].apply(lambda x:  1 if x > 1 else x)

In [24]:
central_df['two_mkt'] = central_df['exact_Evo'] +  central_df['exact_c9'] + central_df['exact_SR']
central_df['two_mkt'] = central_df['two_mkt'].apply(lambda x:  1 if x > 1 else x)

central_df['h_two_mkt'] = central_df['hamming_evo'] +  central_df['hamming_c9'] + central_df['hamming_SR']
central_df['h_two_mkt'] = central_df['h_two_mkt'].apply(lambda x:  1 if x > 1 else x)

In [25]:
central_df['all_mkt'] = central_df['exact_Evo'] * central_df['exact_c9'] * central_df['exact_SR']
central_df['h_all_mkt'] = central_df['hamming_evo'] * central_df['hamming_c9'] * central_df['hamming_SR']


In [26]:
central_df['ag_and_dead_market'] =  central_df['hamming_c9'] + central_df['hamming_SR']
central_df['ag_and_dead_market'] = central_df['ag_and_dead_market'].apply(lambda x:  1 if x > 1 else x)

central_df['ag_and_evo_market'] =  central_df['hamming_evo']

In [27]:
central_df['h_ag_only'] = 1 - central_df['h_two_mkt']

In [38]:
fanfou_df = pd.read_csv('data/fanfou.csv')

In [42]:
fanfou_df['vendor'] = fanfou_df['vendor'].apply(cleanName)


,vendor,date_min,date_max
0,-bigg-balls-,2014-09-22,2014-09-22
1,-euphoria-,2015-04-02,2015-07-07
2,-flip-,2014-12-10,2015-03-19
3,-inanna-,2014-02-02,2015-07-07
4,-tarnation-,2014-12-01,2015-01-15
...,...,...,...
3125,zunidog,2014-02-23,2015-07-07
3126,~bigboy,2014-08-03,2015-07-07
3127,~flatbushzombies~,2014-04-06,2014-04-06
3128,~luxurytransport,2015-06-15,2015-06-15


In [51]:
central_df = central_df.set_index('vendor').join(fanfou_df.set_index('vendor'))

In [63]:
central_df

,exact_Evo,exact_SR,exact_c9,hamming_SR,hamming_evo,hamming_c9,three_mkt,h_three_mkt,two_mkt,h_two_mkt,all_mkt,h_all_mkt,ag_and_dead_market,ag_and_evo_market,h_ag_only,date_min,date_max
vendor,,,,,,,,,,,,,,,,,
"""<i class=""""fa fa-t""",NaN,NaN,NaN,0,0,0,NaN,0,NaN,0,NaN,0,0,0,1,NaN,NaN
-bigg-balls-,1.0,1.0,1.0,1,1,1,1.0,1,1.0,1,1.0,1,1,1,0,2014-09-22,2014-09-22
-euphoria-,NaN,NaN,NaN,0,0,0,NaN,0,NaN,0,NaN,0,0,0,1,2015-04-02,2015-07-07
-flip-,NaN,NaN,NaN,0,0,0,NaN,0,NaN,0,NaN,0,0,0,1,2014-12-10,2015-03-19
-inanna-,NaN,NaN,NaN,0,0,0,NaN,0,NaN,0,NaN,0,0,0,1,2014-02-02,2015-07-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zya,NaN,NaN,NaN,0,0,0,NaN,0,NaN,0,NaN,0,0,0,1,2014-01-09,2014-05-24
~bigboy,NaN,NaN,NaN,0,0,0,NaN,0,NaN,0,NaN,0,0,0,1,2014-08-03,2015-07-07
~flatbushzombies~,NaN,NaN,NaN,0,0,0,NaN,0,NaN,0,NaN,0,0,0,1,2014-04-06,2014-04-06


In [30]:
agora_here_aft_df = central_df[central_df['date_max'] > '2014-12-10']
agora_here_bef_df = central_df[central_df['date_min'] < '2014-11-06']

## Population that surviced Onymous

In [64]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
agora_surv_df = central_df[(central_df['date_max'] > '2015-01-01') &
                           (central_df['date_min'] < '2014-11-06')]
test = agora_surv_df.sum()
colors = ['mediumturquoise', 'darkorange', 'lightgreen']

fig = go.Figure(data=[go.Pie(labels=['A & Dead', 'A Only', 'A & Evo'],
                             values=[test['ag_and_dead_market'], test['h_ag_only'], test['ag_and_evo_market']])])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
fig.show()

# Data to 1

In [65]:

agora_surv_df = central_df[(central_df['date_max'] > '2015-01-01') &
                           (central_df['date_min'] < '2014-11-06')]
test = agora_surv_df.sum()
colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']

fig = go.Figure(data=[go.Pie(labels=['2', 'A', '3', '1'],
                             values=[test['h_two_mkt'] - test['h_three_mkt'] - test['h_all_mkt'], test['h_all_mkt'], test['h_three_mkt'], test['h_ag_only']])])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
fig.show()

# Data to 1


## Population arrived after Onymous

In [66]:
agora_aft_df = central_df[(central_df['date_min'] > '2014-11-06')]
test = agora_aft_df.sum()
# Data to 1
# Data to 1
colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']

fig = go.Figure(data=[go.Pie(labels=['2 Shops', '4 Shops', '3 Shops', '1 Shop'],
                             values=[test['h_two_mkt'] - test['h_three_mkt'] - test['h_all_mkt'], test['h_all_mkt'], test['h_three_mkt'], test['h_ag_only']])])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
fig.show()

# Data to 1


## Population that was there before Onymous

In [34]:
agora_bef_df = central_df[(central_df['date_min'] < '2014-11-06')]
test = agora_bef_df.sum()
# Data to 1
colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']

fig = go.Figure(data=[go.Pie(labels=['2 Shops', '4 Shops', '3 Shops', '1 Shop'],
                             values=[test['h_two_mkt'] - test['h_three_mkt'] - test['h_all_mkt'], test['h_all_mkt'], test['h_three_mkt'], test['h_ag_only']])])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
fig.show()

# Data to 1


## Population that died with Onymous

In [67]:
agora_dead_df = central_df[(central_df['date_max'] < '2014-11-06')]
test = agora_dead_df.sum()
# Data to 1
colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']

fig = go.Figure(data=[go.Pie(labels=['2 Shops', '4 Shops', '3 Shops', '1 Shop'],
                             values=[test['h_two_mkt'] - test['h_three_mkt'] - test['h_all_mkt'], test['h_all_mkt'], test['h_three_mkt'], test['h_ag_only']])])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
fig.show()

# Data to 1


In [68]:
agora_dead_df = central_df[(central_df['date_max'] < '2014-11-06')]
test = agora_dead_df.sum()
# Data to 1
colors = ['mediumturquoise', 'darkorange', 'lightgreen']

fig = go.Figure(data=[go.Pie(labels=['A & Dead', 'A Only', 'A & Evo'],
                             values=[test['ag_and_dead_market'], test['h_ag_only'], test['ag_and_evo_market']])])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
fig.show()

# Data to 1

In [73]:
temp = central_df.drop(columns=['exact_Evo', 'exact_SR', 'exact_c9', 'three_mkt', 'two_mkt', 'all_mkt']).dropna()

In [78]:
temp.columns

Index(['hamming_SR', 'hamming_evo', 'hamming_c9', 'h_three_mkt', 'h_two_mkt',
       'h_all_mkt', 'ag_and_dead_market', 'ag_and_evo_market', 'h_ag_only',
       'date_min', 'date_max'],
      dtype='object')

In [81]:
temp.drop(columns=['hamming_SR', 'hamming_evo', 'hamming_c9', 'h_three_mkt','h_two_mkt', 'h_all_mkt']).to_csv('data/pie_graph_df.csv')